In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date, StringType as Str, TimestampType

In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
spark = create_spark_session()

In [3]:
log_input_data = "data/more/log_data/2018-11-01-events.json"
    #or data/more/song-data/*.json"
    
    # read song data file
logSchema = R([
    Fld("artist",Str()),
    Fld("auth",Str()),
    Fld("firstName",Str()),
    Fld("gender",Str()),
    Fld("itemInSession",Dbl()),
    Fld("lastName",Str()),
    Fld("artist_latitude",Dbl()),
    Fld("length",Dbl()),
    Fld("level",Str()),
    Fld("location",Str()),
    Fld("method",Str()),
    Fld("page",Str()),
    Fld("registration",Dbl()),
    Fld("sessionId",Str()),
    Fld("song",Str()),
    Fld("status",Dbl()),
    Fld("ts",Dbl()),
    Fld("userAgent",Str()),
    Fld("userId",Str())
    
])
df = spark.read.format("json").load(log_input_data, schema=logSchema)

In [4]:
log_input_data

'data/more/log_data/2018-11-01-events.json'

In [5]:
print(df.count())
df.printSchema()

15
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: double (nullable = true)
 |-- lastName: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: string (nullable = true)
 |-- song: string (nullable = true)
 |-- status: double (nullable = true)
 |-- ts: double (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [58]:
df.show()

+--------------------+---------+---------+------+-------------+--------+---------------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-----------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|artist_latitude|   length|level|            location|method|    page|     registration|sessionId|                song|status|               ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-----------------+--------------------+------+
|                null|Logged In|   Walter|     M|          0.0|    Frye|           null|     null| free|San Francisco-Oak...|   GET|    Home|1.540919166796E12|       38|                null| 200.0|1.541105830796E12|"Mozilla/5.0 (Mac...|    39|
|                null|Lo

In [6]:
# filter by actions for song plays

df_filtered=df[df['page']=="NextSong"]

In [7]:
df_filtered.count()

11

In [8]:
# define output bucket und AWS S3
#output = "s3a://sparkifyphil.s3.us-east-2.amazonaws.com/output/song_data/"
output="partitioned"
# create schema-on-read for table "songs"
df_filtered.createOrReplaceTempView("logs")

In [62]:
# extract columns for users table    
users_table = spark.sql("""
    SELECT  
        DISTINCT userId AS user_id,
        firstName AS first_name,
        lastName AS last_name, 
        gender, 
        level
    FROM logs  

""")




In [63]:
users_table.write.parquet(output+"-users_table")

AnalysisException: 'path file:/home/workspace/partitioned-users_table already exists.;'

In [9]:


# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: x/1000, Dbl())
df_filtered = df_filtered.withColumn('start_time', get_timestamp('ts'))



In [10]:
df_filtered[['start_time']].show()
df_filtered[['ts']].show()


+----------------+
|      start_time|
+----------------+
|1.541106106796E9|
|1.541106352796E9|
|1.541106496796E9|
|1.541106673796E9|
|1.541107053796E9|
|1.541107493796E9|
|1.541107734796E9|
|1.541108520796E9|
|1.541109125796E9|
|1.541109325796E9|
|1.541110994796E9|
+----------------+

+-----------------+
|               ts|
+-----------------+
|1.541106106796E12|
|1.541106352796E12|
|1.541106496796E12|
|1.541106673796E12|
|1.541107053796E12|
|1.541107493796E12|
|1.541107734796E12|
|1.541108520796E12|
|1.541109125796E12|
|1.541109325796E12|
|1.541110994796E12|
+-----------------+



In [18]:
# create datetime column from original timestamp column
#get_datetime = udf(lambda x: from_unixtime(x), TimestampType())
#df = df.withColumn('datetime', from_unixtime('start_time'))

df_with_time = df_filtered.withColumn("TimestampType",F.to_timestamp(df_filtered["start_time"]))
#df2.show(n=2,truncate=False)
df_with_time[['TimestampType']].show()

+--------------------+
|       TimestampType|
+--------------------+
|2018-11-01 21:01:...|
|2018-11-01 21:05:...|
|2018-11-01 21:08:...|
|2018-11-01 21:11:...|
|2018-11-01 21:17:...|
|2018-11-01 21:24:...|
|2018-11-01 21:28:...|
|2018-11-01 21:42:...|
|2018-11-01 21:52:...|
|2018-11-01 21:55:...|
|2018-11-01 22:23:...|
+--------------------+



In [23]:
#df_with_time = df_with_time.withColumn("hour",F.hour(df_with_time["TimestampType"]))
#df_with_time[['hour']].show()
#df_with_time = df_with_time.withColumn("day",F.dayofmonth(df_with_time["TimestampType"]))
#df_with_time[['day']].show()
#df_with_time = df_with_time.withColumn("week",F.weekofyear(df_with_time["TimestampType"]))
#df_with_time[['week']].show()
#df_with_time = df_with_time.withColumn("month",F.month(df_with_time["TimestampType"]))
#df_with_time[['month']].show()
df_with_time = df_with_time.withColumn("year",F.year(df_with_time["TimestampType"]))
df_with_time[['year']].show()
df_with_time = df_with_time.withColumn("weekday",F.dayofweek(df_with_time["TimestampType"]))
df_with_time[['weekday']].show()

+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
+----+

+-------+
|weekday|
+-------+
|      5|
|      5|
|      5|
|      5|
|      5|
|      5|
|      5|
|      5|
|      5|
|      5|
|      5|
+-------+



In [25]:
df_with_time.createOrReplaceTempView("logs_with_time")
# extract columns for times table    
time_table = spark.sql("""
    SELECT  
        TimestampType AS start_time,
        hour,
        day,
        week,
        month
        year, 
        weekday
    FROM logs_with_time  

""")
time_table.write.parquet(output+"-time_table")

In [29]:
 # read in song data to use for songplays table
song_df = spark.read.parquet(output + "-song_table")
song_df.createOrReplaceTempView("songs")
song_df.show(5)

+------------------+---------+---------+----+------------------+
|           song_id|    title| duration|year|         artist_id|
+------------------+---------+---------+----+------------------+
|SOCIWDW12A8C13D406|Soul Deep|148.03546|1969|ARMJAGH1187FB546F3|
+------------------+---------+---------+----+------------------+



In [34]:
#generate a new column on the  logs to hold the unique id
df_with_time=df_with_time.withColumn("id", F.monotonically_increasing_id() + 123)

In [36]:
df_with_time[['id']].show()

+---+
| id|
+---+
|123|
|124|
|125|
|126|
|127|
|128|
|129|
|130|
|131|
|132|
|133|
+---+



In [43]:
df_with_time.createOrReplaceTempView("logs")
# extract columns from joined song and log datasets to create songplays table 
songplays_table = spark.sql("""
    SELECT
    logs.id AS songplay_id , 
    logs.start_time AS start_time, 
    logs.userId AS user_id, 
    logs.level AS level, 
    songs.song_id AS song_id, 
    songs.artist_id, 
    logs.sessionId AS session_id, 
    logs.location, 
    logs.userAgent AS user_agent,
    logs.year,
    logs.month
    FROM logs,songs
    WHERE songs.title = logs.song
    

""")


In [44]:
songplays_table.show()

+-----------+----------+-------+-----+-------+---------+----------+--------+----------+----+-----+
|songplay_id|start_time|user_id|level|song_id|artist_id|session_id|location|user_agent|year|month|
+-----------+----------+-------+-----+-------+---------+----------+--------+----------+----+-----+
+-----------+----------+-------+-----+-------+---------+----------+--------+----------+----+-----+



In [46]:
songplays_table.write.partitionBy("year", "month").parquet(output + "-songplays_table")

In [ ]:
logSchema = R([
    Fld("artist",Str()),
    Fld("auth",Str()),
    Fld("firstName",Str()),
    Fld("gender",Str()),
    Fld("itemInSession",Dbl()),
    Fld("lastName",Str()),
    Fld("artist_latitude",Dbl()),
    Fld("length",Dbl()),
    Fld("level",Str()),
    Fld("location",Str()),
    Fld("method",Str()),
    Fld("page",Str()),
    Fld("registration",Dbl()),
    Fld("sessionId",Str()),
    Fld("song",Str()),
    Fld("status",Dbl()),
    Fld("ts",Dbl()),
    Fld("userAgent",Str()),
    Fld("userId",Str())